In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time
from time import time as timer

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

/home/cfchen/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    'gpu': 0.85
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.05771303176879883s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

## With Video Resizing

In [7]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('temp.avi',fourcc, 30.0, (960, 540))     # resize image half

print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

start = timer()

count = 0
while(True):
    ret, frame = capture.read() 

    if ret == True:
        
        count += 1
        
        frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)  # resize image half
                    
        results = tfnet.return_predict(frame)
        
        #print(results[0])
        #print(results[0]['confidence'])
        #print(results[0]['label'])
        
        print('Frame Number {}: Found {} boxes for {}'.format(count - 1, len(results), 'img'))
        
        for result in results:
            '''
            if result['label'] != 'car, 3' and result['label'] != 'truck, 8':
                continue            
            '''
            
            if (result['confidence'] <= 0.4) or (result['label'] != 'car, 3' and result['label'] != 'truck, 8'):
                continue
            
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 2)
            frame = cv2.rectangle(frame, tl, br, (255, 255, 0), 7)
            #frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 255), 2)
            frame = cv2.putText(frame, label + ',' + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 255), 2)
            
            #print(result['label'])
            
            '''
            Label_Confidence = '{}, {:.2f}'.format(result['label'], result['confidence'])
            TLXY = '{}, {}'.format(result['topleft']['x'], result['topleft']['y'])
            BRXY = '{}, {}'.format(result['bottomright']['x'] , result['bottomright']['y'])
            print("{}, {}, {}, {}".format(count - 1 , Label_Confidence, TLXY, BRXY))
            '''

            with open('outputEis.csv', 'a') as f:
                Label_Confidence = '{}, {:.2f}'.format(result['label'], result['confidence'])
                TLXY = '{}, {}'.format(result['topleft']['x'], result['topleft']['y'])
                BRXY = '{}, {}'.format(result['bottomright']['x'] , result['bottomright']['y'])
                print("{}, {}, {}, {}".format(count - 1 , Label_Confidence, TLXY, BRXY), file=f)
            #f.close()
        
        out.write(frame)
        cv2.imshow('frame', frame)

        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break
            
    else:
        break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nTotal elapsed time = ' + str(timer() - start) + ' s\n')

Video Dim: 1920.0  X  1080.0
Frame Number 0: Found 8 boxes for img
Frame Number 1: Found 10 boxes for img
Frame Number 2: Found 10 boxes for img
Frame Number 3: Found 9 boxes for img
Frame Number 4: Found 10 boxes for img
Frame Number 5: Found 11 boxes for img
Frame Number 6: Found 10 boxes for img
Frame Number 7: Found 8 boxes for img
Frame Number 8: Found 8 boxes for img
Frame Number 9: Found 8 boxes for img
Frame Number 10: Found 10 boxes for img
Frame Number 11: Found 10 boxes for img
Frame Number 12: Found 7 boxes for img
Frame Number 13: Found 6 boxes for img
Frame Number 14: Found 7 boxes for img
Frame Number 15: Found 7 boxes for img
Frame Number 16: Found 7 boxes for img
Frame Number 17: Found 7 boxes for img
Frame Number 18: Found 7 boxes for img
Frame Number 19: Found 7 boxes for img
Frame Number 20: Found 8 boxes for img
Frame Number 21: Found 8 boxes for img
Frame Number 22: Found 8 boxes for img
Frame Number 23: Found 8 boxes for img
Frame Number 24: Found 8 boxes for img

Frame Number 208: Found 6 boxes for img
Frame Number 209: Found 6 boxes for img
Frame Number 210: Found 6 boxes for img
Frame Number 211: Found 6 boxes for img
Frame Number 212: Found 6 boxes for img
Frame Number 213: Found 6 boxes for img
Frame Number 214: Found 6 boxes for img
Frame Number 215: Found 6 boxes for img
Frame Number 216: Found 4 boxes for img
Frame Number 217: Found 4 boxes for img
Frame Number 218: Found 4 boxes for img
Frame Number 219: Found 4 boxes for img
Frame Number 220: Found 4 boxes for img
Frame Number 221: Found 4 boxes for img
Frame Number 222: Found 4 boxes for img
Frame Number 223: Found 4 boxes for img
Frame Number 224: Found 4 boxes for img
Frame Number 225: Found 4 boxes for img
Frame Number 226: Found 4 boxes for img
Frame Number 227: Found 4 boxes for img
Frame Number 228: Found 5 boxes for img
Frame Number 229: Found 6 boxes for img
Frame Number 230: Found 6 boxes for img
Frame Number 231: Found 4 boxes for img
Frame Number 232: Found 5 boxes for img
